In [ ]:
# Koala Detection Project

#This notebook covers the entire workflow from installing necessary libraries to training and exporting an object detection model for Koala detection using TensorFlow Object Detection API.

## Step 1: Install Necessary Libraries

#```python
!pip install tensorflow
!pip install tf_slim
!pip install pycocotools
!pip install lvis
!pip install contextlib2
!pip install matplotlib
!pip install pillow

# Clone the TensorFlow models repository
!git clone --depth 1 https://github.com/tensorflow/models

# Install the Object Detection API
%cd models/research
!protoc object_detection/protos/*.proto --python_out=.
!cp object_detection/packages/tf2/setup.py .
!pip install .

# Verify the installation
!python object_detection/builders/model_builder_tf2_test.py


In [ ]:
import os
import urllib.request
import tarfile

# Define the model to download
MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
BASE_DIR = r'C:\Users\arron\OneDrive\Documents\UTS\Post Graduate\Semester 2\42028 Deep Learning and Convolutional Neural Networks\Assignments\Assignment 3\Part E'
MODEL_DIR = os.path.join(BASE_DIR, 'koala_model')
MODEL_URL = f'http://download.tensorflow.org/models/object_detection/tf2/20200711/{MODEL_NAME}.tar.gz'

# Create a directory for the model
os.makedirs(MODEL_DIR, exist_ok=True)

# Download the model
model_path = os.path.join(MODEL_DIR, f'{MODEL_NAME}.tar.gz')
urllib.request.urlretrieve(MODEL_URL, model_path)

# Extract the model
with tarfile.open(model_path) as tar:
    tar.extractall(path=MODEL_DIR)

print(f"Model {MODEL_NAME} downloaded and extracted successfully!")


In [ ]:
import tensorflow as tf
import os
import glob
import xml.etree.ElementTree as ET
from object_detection.utils import dataset_util

# Define the label map
label_map = {
    "background": 0,
    "koalas": 1
}

def create_tf_example(annotation_path, image_path):
    with open(annotation_path) as fid:
        xml_str = fid.read()
    xml = ET.fromstring(xml_str)
    filename = xml.find('filename').text
    img_path = os.path.join(image_path, filename)
    if not os.path.exists(img_path):
        print(f"Image file {img_path} not found.")
        return None
    with tf.io.gfile.GFile(img_path, 'rb') as fid:
        encoded_jpg = fid.read()
    size = xml.find('size')
    width = int(size.find('width').text)
    height = int(size.find('height').text)

    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for member in xml.findall('object'):
        class_name = member.find('name').text
        if class_name in label_map:
            class_id = label_map[class_name]
        else:
            continue

        xmin = float(member.find('bndbox').find('xmin').text) / width
        xmax = float(member.find('bndbox').find('xmax').text) / width
        ymin = float(member.find('bndbox').find('ymin').text) / height
        ymax = float(member.find('bndbox').find('ymax').text) / height
        xmins.append(xmin)
        xmaxs.append(xmax)
        ymins.append(ymin)
        ymaxs.append(ymax)
        classes_text.append(class_name.encode('utf8'))
        classes.append(class_id)

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename.encode('utf8')),
        'image/source_id': dataset_util.bytes_feature(filename.encode('utf8')),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature('jpeg'.encode('utf8')),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

def create_tf_record(output_path, annotation_dir, image_dir):
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    writer = tf.io.TFRecordWriter(output_path)
    annotation_paths = glob.glob(os.path.join(annotation_dir, '*.xml'))
    for annotation_path in annotation_paths:
        tf_example = create_tf_example(annotation_path, image_dir)
        if tf_example is not None:
            writer.write(tf_example.SerializeToString())
    writer.close()

# Define paths
output_path = r'C:\Users\arron\OneDrive\Documents\UTS\Post Graduate\Semester 2\42028 Deep Learning and Convolutional Neural Networks\Assignments\Assignment 3\Part E\koala_dataset\koala annotated dataset\train.record'
annotation_dir = r'C:\Users\arron\OneDrive\Documents\UTS\Post Graduate\Semester 2\42028 Deep Learning and Convolutional Neural Networks\Assignments\Assignment 3\Downloaded Dataset\koala annotated dataset\Annotations'
image_dir = r'C:\Users\arron\OneDrive\Documents\UTS\Post Graduate\Semester 2\42028 Deep Learning and Convolutional Neural Networks\Assignments\Assignment 3\Downloaded Dataset\koala annotated dataset\images'

# Create TFRecord
create_tf_record(output_path, annotation_dir, image_dir)
print("TFRecord created successfully!")


In [ ]:
import os
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

# Define paths
model_dir = os.path.join(BASE_DIR, 'koala_model/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8')
pipeline_config_path = os.path.join(model_dir, 'pipeline.config')
label_map_path = os.path.join(BASE_DIR, 'koala_dataset/koala annotated dataset/label_map.pbtxt')
train_record_path = os.path.join(BASE_DIR, 'koala_dataset/koala annotated dataset/train.record')

# Load pipeline config and modify it
configs = config_util.get_configs_from_pipeline_file(pipeline_config_path)
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(pipeline_config_path, "r") as f:
    proto_str = f.read()
    text_format.Merge(proto_str, pipeline_config)

pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [train_record_path]
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [train_record_path]
pipeline_config.train_input_reader.label_map_path = label_map_path
pipeline_config.eval_input_reader[0].label_map_path = label_map_path
pipeline_config.model.ssd.num_classes = 1  # Assuming one class (koala)

# Save updated pipeline config
config_util.save_pipeline_config(pipeline_config, model_dir)
print("Pipeline configuration updated successfully!")


BASH CODE NEXT

In [ ]:
# Ensure you're in the right directory
cd models/research

# Train the model
python model_main_tf2.py \
    --model_dir="C:\Users\arron\OneDrive\Documents\UTS\Post Graduate\Semester 2\42028 Deep Learning and Convolutional Neural Networks\Assignments\Assignment 3\Part E\koala_model\ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8" \
    --pipeline_config_path="C:\Users\arron\OneDrive\Documents\UTS\Post Graduate\Semester 2\42028 Deep Learning and Convolutional Neural Networks\Assignments\Assignment 3\Part E\koala_model\ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8\pipeline.config" \
    --num_train_steps=2000 \
    --sample_1_of_n_eval_examples=1 \
    --alsologtostderr


In [ ]:
!python model_main_tf2.py \
    --model_dir="C:\Users\arron\OneDrive\Documents\UTS\Post Graduate\Semester 2\42028 Deep Learning and Convolutional Neural Networks\Assignments\Assignment 3\Part E\koala_model\ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8" \
    --pipeline_config_path="C:\Users\arron\OneDrive\Documents\UTS\Post Graduate\Semester 2\42028 Deep Learning and Convolutional Neural Networks\Assignments\Assignment 3\Part E\koala_model\ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8\pipeline.config" \
    --checkpoint_dir="C:\Users\arron\OneDrive\Documents\UTS\Post Graduate\Semester 2\42028 Deep Learning and Convolutional Neural Networks\Assignments\Assignment 3\Part E\koala_model\ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8" \
    --eval_training_data=True \
    --alsologtostderr


In [ ]:
!python exporter_main_v2.py \
    --input_type image_tensor \
    --pipeline_config_path="C:\Users\arron\OneDrive\Documents\UTS\Post Graduate\Semester 2\42028 Deep Learning and Convolutional Neural Networks\Assignments\Assignment 3\Part E\koala_model\ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8\pipeline.config" \
    --trained_checkpoint_dir="C:\Users\arron\OneDrive\Documents\UTS\Post Graduate\Semester 2\42028 Deep Learning and Convolutional Neural Networks\Assignments\Assignment 3\Part E\koala_model\ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8" \
    --output_directory="C:\Users\arron\OneDrive\Documents\UTS\Post Graduate\Semester 2\42028 Deep Learning and Convolutional Neural Networks\Assignments\Assignment 3\Part E\koala_model\ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8\exported_model"
